# 第8章 深層学習による画像認識とその仕組みを知ろう

ここでは、深層学習を学ぶうえで必要なプログラムを実行していく流れを学んでいきます。  
Google Colaboratory上で実行する場合、”ランタイム”から”ランタイムのタイプの変更”を開きGPUが選択されている事を確認して下さい。

※エラーが出る場合は、Clear Output を行うことで正常に動く場合があります。

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter8

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements.txt

## 8-6 学習データとしての画像の構造を理解しよう


In [ ]:
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

# cifar10と言うデータセットを使う。
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print("x_train.shape: ",x_train.shape)
print("y_train.shape: ",y_train.shape)
print("x_test.shape: ",x_test.shape)
print("y_test.shape: ",y_test.shape)

In [ ]:
print("shape: ",x_train[0].shape)
print(x_train[0])

In [ ]:
#学習データの最初の画像を表示
plt.imshow(x_train[0])
plt.show()

In [ ]:
#学習データの一番最初の画像のラベルを表示
print(y_train[0])

#学習データ、テストデータのラベルが取りうる値を列挙
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
#ラベルの番号と名前を対応付ける。例えばラベルが６ならlabel_names[6]で、frogになる。
label_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

plt.figure(figsize=(10,5))
for index in range(10):
    img = x_train[index]
    label = label_names[y_train[index][0]]
    plt.subplot(2,5,index+1)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img)

## 8-7 深層学習ライブラリを使ってゼロから画像データを学習してみよう


In [ ]:
from tensorflow.keras.utils import to_categorical

#画像の各ピクセルの値が0~1の間の値を取るようにする
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

#ラベルのOnehot encodingを行う
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

#モデルの構築
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu', name="conv2d_1"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu', name="conv2d_2"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(label_names), activation='softmax'))

#モデルの概要を表示
print(model.summary())

model.compile(optimizer = optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs=20

#学習開始
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)

In [ ]:
#正答率を計算
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred,axis = 1) 
test_loss, test_acc = model.evaluate(x_test, y_test)

print(test_acc)

## 8-8 学習した結果を評価しよう

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='g', label="Validation Loss")
legend = ax[0].legend()

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='g', label="Validation Accuracy")
legend = ax[1].legend()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns

y_pred = model.predict(x_test)
#y_predは、各クラスになる確率が入っているのでそれぞれで最大値だけを取る
y_pred_classes = np.argmax(y_pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)
cf_matrix = confusion_matrix(y_true, y_pred_classes)

In [ ]:
plt.figure(figsize=(13, 13))
c = sns.heatmap(cf_matrix, annot=True,fmt="d")
c.set(xticklabels=label_names, yticklabels=label_names)
plt.plot()

## 8-9 学習したネットワークが見ている「特徴」を可視化してみよう

In [ ]:
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Model
from matplotlib import colors
from PIL import Image

def grad_cam_image(model, layer_name, image):
  
  with tf.GradientTape() as tape:
    layer = model.get_layer(layer_name)

    #出力を、普通の出力（１０個に分類する出力）と、layer_nameで指定した層の出力の２つにする。
    tmpModel = Model([model.inputs], [model.output, layer.output])
    #model_outは入力した画像の分類結果。
    #layer_outはlayer_nameで指定した層の出力
    model_out, layer_out = tmpModel(np.array([image]))

    #モデルの分類結果で一番高い確率をclass_outに格納
    class_out = model_out[:, np.argmax(model_out[0])]
    #出力から、指定した層までの勾配を計算
    grads = tape.gradient(class_out, layer_out)
    #勾配の平均を取る. Global Average Poolingと同じこと。
    pooled_grads = K.mean(grads, axis=(0, 1, 2))


  #計算した勾配の平均を指定した層の出力にかける
  heatmap = tf.multiply(pooled_grads, layer_out)
  #チャンネル毎に足し合わせる
  heatmap = tf.reduce_sum(heatmap, axis=-1)
  #マイナスの値を取らないようにする。ReLuと同じ処理
  heatmap = np.maximum(heatmap, 0)
  #0~1の値に収める
  heatmap = heatmap/heatmap.max()

  #見やすい画像にする
  return_image = np.asarray(Image.fromarray(heatmap[0]).resize(image.shape[:2])) * 255
  colormap = plt.get_cmap('jet')
  return_image = return_image.reshape(-1)
  return_image = np.array([colormap(int(np.round(pixel)))[:3] for pixel in return_image]).reshape(image.shape)
  return_image = image * 0.5 + return_image * 0.5

  return return_image

In [ ]:
[layer.name for layer in model.layers]

In [ ]:
grad_cam = grad_cam_image(model, "conv2d_2", x_train[0])

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(grad_cam)

plt.subplot(1,2,2)
plt.imshow(x_train[0])
plt.show()

## 8-10 学習したネットワークの中身を可視化してみよう

In [ ]:
def show_filters(model, layer_name):
    target_layer = model.get_layer(layer_name).get_weights()[0]
    filter_num = target_layer.shape[3]

    plt.figure(figsize=(15, 10))
    for i in range(filter_num):
        plt.subplot(int(filter_num/6) + 1, 6, i+1)
        plt.title('filter %d' % i)
        plt.axis('off')
        plt.imshow(target_layer[ :, :, 0, i], cmap="gray") 
    plt.show()
    
show_filters(model, "conv2d_1")

In [ ]:
from tensorflow.keras.models import Model

#モデルと画像を渡すと、各畳み込み層での出力を画像として表示する。
def layer_outputs(model, image):
    #畳み込み層のみ抽出
    _model = Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers if type(layer) is Conv2D])

    #渡された画像の分類を実行
    conv_outputs = _model.predict(np.array([image]))
    
    def show_images(output, title):
        output = output[0]
        filter_num = output.shape[2]
        
        fig = plt.figure(figsize=(20, 15))
        fig.suptitle(title, size=15)
        for i in range(filter_num):
            plt.subplot(int(filter_num/8) + 1, 8, i+1)
            plt.title('filter %d' % i)
            plt.axis('off')
            plt.imshow(output[:,:,i])
    
    #畳み込み層毎に画像を出力
    for i, output in enumerate(conv_outputs):
        title = "Conv layer number %d" % (i + 1)
        show_images(output, title)
        
layer_outputs(model, x_train[0])